# StrongholdNet: Train an RNN (LSTM) to navigate through a Stronghold

The idea is that we interpret the (shortest) path from any room in the stronghold to the portal room as *sequential data* that we feed to an RNN.

In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras as K
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from dataset_rnn import parse_tree_generator, print_stronghold_tree
from anytree import Node, RenderTree, Walker
from anytree.search import find_by_attr
import random

In [10]:
df = pd.read_csv('100k_dataset_rnn.csv', delimiter=' ')

In [11]:
df.head(24)

,stronghold,room,downwards,orientation,parent_room,parent_exit,child_room_1,child_room_2,child_room_3,child_room_4,child_room_5,exit
0,0,RightTurn,0,S,Stairs,1,Stairs,None,None,None,None,0
1,0,Stairs,0,S,Corridor,2,RightTurn,None,None,None,None,0
2,0,Corridor,0,W,FiveWayCrossing,1,Corridor,Stairs,RightTurn,None,None,1
3,0,Corridor,1,W,Corridor,1,LeftTurn,Corridor,None,None,None,2
4,0,Corridor,1,S,Corridor,2,Corridor,None,None,None,None,1
5,0,Corridor,1,S,Corridor,1,RightTurn,None,None,None,None,1
6,0,RightTurn,1,S,Corridor,1,PortalRoom,None,None,None,None,1
7,1,PrisonHall,0,N,SquareRoom,1,Corridor,None,None,None,None,0
8,1,SquareRoom,0,N,ChestCorridor,1,PrisonHall,RightTurn,ChestCorridor,None,None,0
9,1,ChestCorridor,0,N,RightTurn,1,SquareRoom,None,None,None,None,0


The problem is that we have to structure our data such that in each time step the model gets to know where we went in the last step.

In [66]:
# one-hot encode
cols = [
        'room',
        'downwards',
        'orientation',
        'parent_room',
        'parent_exit',
        'child_room_1',
        'child_room_2',
        'child_room_3',
        'child_room_4',
        'child_room_5']
onehot = ColumnTransformer([("one-hot", OneHotEncoder(), cols)], remainder='passthrough')
onehot.fit(df)
df_onehot = pd.DataFrame(onehot.transform(df).toarray(), index=df.index, columns=pd.Index(onehot.get_feature_names()))

We need to group our sequences by stronghold. Notice how each sequence starts at a random room and ends with a room that has the `PortalRoom` as one of its children.

In [64]:
def sequencelify(df, window):
    for _, s in df.groupby('stronghold'):
        s = s.drop('stronghold', axis=1).to_numpy()
        for t in range(s.shape[0] - window + 1):
            X = s[t:t+window, :-1]
            y = np.array(s[t+window-1, -1])
            X = X.reshape(1, window, X.shape[1])
            y = y.reshape(1, 1)
            yield X, y

In [69]:
# just checking if generator works
count = 0
for x, y in sequencelify(df, 4):
    print(x)
    print(y)
    print()
    count += 1
    if count == 10:
        break

[[['RightTurn' 0 'S' 'Stairs' 1 'Stairs' 'None' 'None' 'None' 'None']
  ['Stairs' 0 'S' 'Corridor' 2 'RightTurn' 'None' 'None' 'None' 'None']
  ['Corridor' 0 'W' 'FiveWayCrossing' 1 'Corridor' 'Stairs' 'RightTurn'
   'None' 'None']
  ['Corridor' 1 'W' 'Corridor' 1 'LeftTurn' 'Corridor' 'None' 'None'
   'None']]]
[[2]]

[[['Stairs' 0 'S' 'Corridor' 2 'RightTurn' 'None' 'None' 'None' 'None']
  ['Corridor' 0 'W' 'FiveWayCrossing' 1 'Corridor' 'Stairs' 'RightTurn'
   'None' 'None']
  ['Corridor' 1 'W' 'Corridor' 1 'LeftTurn' 'Corridor' 'None' 'None'
   'None']
  ['Corridor' 1 'S' 'Corridor' 2 'Corridor' 'None' 'None' 'None' 'None']]]
[[1]]

[[['Corridor' 0 'W' 'FiveWayCrossing' 1 'Corridor' 'Stairs' 'RightTurn'
   'None' 'None']
  ['Corridor' 1 'W' 'Corridor' 1 'LeftTurn' 'Corridor' 'None' 'None'
   'None']
  ['Corridor' 1 'S' 'Corridor' 2 'Corridor' 'None' 'None' 'None' 'None']
  ['Corridor' 1 'S' 'Corridor' 1 'RightTurn' 'None' 'None' 'None' 'None']]]
[[1]]

[[['Corridor' 1 'W' 'Corridor

In [71]:
#X_train, X_test, y_train, y_test = train_test_split(
#        df_onehot.drop('exit', axis=1),
#        df_onehot['exit'],
#        test_size=0.1,
#        random_state=1337,
#        shuffle=False)

In [ ]:
#print("X_train:", X_train.shape)
#print("y_train:", y_train.shape)
#print("X_test:", X_test.shape)
#print("y_test:", y_test.shape)

In [ ]:
#X_train_np = X_train.to_numpy()
#n_samples = X_train_np.shape[0]
#n_features = X_train_np.shape[1]
#X_train_np = X_train_np.reshape(n_samples, 1, n_features)
#y_train_np = y_train.to_numpy()

In [70]:
df_train, df_test = train_test_split(
        df_onehot,
        test_size=0.1,
        random_state=1337,
        shuffle=False)

In [72]:
print("df_train:", df_train.shape)
print("df_test:", df_test.shape)

df_train: (631518, 97)
df_test: (70169, 97)


In [76]:
n_samples = df_train.shape[0]
n_features = df_train.shape[1] - 2

In [79]:
model = K.Sequential()
model.add(K.layers.LSTM(
        64,
        batch_input_shape=(1, None, n_features),
        return_sequences=False,
        stateful=False))
model.add(K.layers.Dense(
        6,
        activation='softmax'))
model.compile(
        optimizer='adam',
        loss=K.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 64)                40960     
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 390       
Total params: 41,350
Trainable params: 41,350
Non-trainable params: 0
_________________________________________________________________


In [81]:
model.fit(
        sequencelify(df_train, 4),
        epochs=10,
        steps_per_epoch=10000)

Epoch 1/10
10000/10000 [==============================] - 18s 2ms/step - loss: 0.3278 - accuracy: 0.0341
Epoch 2/10
10000/10000 [==============================] - 18s 2ms/step - loss: 0.2975 - accuracy: 0.0317
Epoch 3/10
 4360/10000 [============>.................] - ETA: 10s - loss: 0.2917 - accuracy: 0.0342

KeyboardInterrupt: 

In [ ]:
#model.save("rnn_2.keras")
#model = K.models.load_model("rnn_2.keras")

In [ ]:
X_test_np = X_test.to_numpy()
n_test_samples = X_test_np.shape[0]
X_test_np = X_test_np.reshape(n_test_samples, 1, n_features)
y_test_np = y_test.to_numpy()

In [ ]:
model.evaluate(X_test_np, y_test_np)

In [ ]:
g = parse_tree_generator('100k_strongholds_test.txt')
root = next(g)

In [ ]:
def evaluate_nav(room: Node, model: K.Model, onehot: OneHotEncoder, n_rooms=0):
    if len([ v for v in room.children if v.name == 'PortalRoom' ]) > 0:
        return n_rooms
    
    if n_rooms > 30:
        #print("giving up")
        return False
    
    if room.name in ['Library', 'SmallCorridor']:
        #print("bad nav")
        return False
    
    #print("room", room.name)
    x = pd.DataFrame([(
            room.name,
            room.orientation,
            room.parent.name,
            room.exit,
            *([c.name for c in room.children] + ['None'] * (5 - len(room.children))), -1)],
            columns=df.columns)
    x_onehot = pd.DataFrame(onehot.transform(x).toarray(), columns=pd.Index(onehot.get_feature_names()))
    x_onehot.drop('exit', axis=1, inplace=True)
    x_np = x_onehot.to_numpy().reshape(1, 1, n_features)
    y_hat = model.predict(x_np)
    exit_hat = y_hat.argmax(axis=-1)[0]
    #print("exit_hat", exit_hat)
    
    return evaluate_nav([room.parent, *room.children][exit_hat], model, onehot, n_rooms + 1)

total = 0
hits = 0
for root in parse_tree_generator('100k_strongholds_test.txt'):
    #print_stronghold_tree(root)
    root = root.children[0]
    n_rooms = evaluate_nav(root, model, onehot)
    #(upwards, common, downwards) = Walker().walk(root, find_by_attr(root, 'PortalRoom'))
    #delta = n_rooms - len(downwards)
    total += 1
    if n_rooms != False:
        hits += 1
        print("n_rooms:", n_rooms)
        print("total:", total)
        print("hits:", hits)
        print("ratio:", hits / total)
        print()